<center>
<h4>CDS 110, Lecture 4a</h4>
<font color=blue><h1>Dynamics and State Feedback Control of a Predator-Prey Model</h1></font>
<h3>Richard M. Murray, Winter 2024</h3>
</center>

[Open in Google Colab](https://colab.research.google.com/drive/1yMOSRNDDNtm-TJGMXX3NS7F4XybOuch-)

In this lecture we describe the use of state space control concepts to analyze and stabilize the dynamics of a nonlinear model of a predator-prey system.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
try:
  import control as ct
  print("python-control", ct.__version__)
except ImportError:
  !pip install control
  import control as ct

## Predator-Prey System Model

We consider a predator-prey system, in which a predator species (lynxes) interacts with a prey species (hares):

<center>
    <img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/predprey-photo.png" alt="predprey-photo" width=400>
    &nbsp;&nbsp;
    <img src="https://www.cds.caltech.edu/~murray/courses/cds110/sp2024/predprey-graph.png" alt="predprey-photo" width=480>
</center>

The graph on the right shows the populations of hares and lynxes between 1845 and 1935 in a section of the Canadian Rockies (MacLulich, 1937).

In [ ]:
# Define the dynamics for the predator-prey system (no input)
predprey_params = {'r': 1.6, 'd': 0.56, 'b': 0.6, 'k': 125, 'a': 3.2, 'c': 50}
def predprey_update(t, x, u, params):
    """Predator prey dynamics"""
    r, d, b, k, a, c = map(params.get, ['r', 'd', 'b', 'k', 'a', 'c'])
    u = np.clip(u, -r, r)

    # Dynamics for the system
    dx0 = (r + u[0]) * x[0] * (1 - x[0]/k) - a * x[1] * x[0]/(c + x[0])
    dx1 = b * a * x[1] * x[0] / (c + x[0]) - d * x[1]

    return np.array([dx0, dx1])

# Create a nonlinear I/O system
predprey = ct.nlsys(
    predprey_update, name='predprey', params=predprey_params,
    states=['H', 'L'], inputs='u', outputs=['H', 'L'])

### Open loop dynamics

The open loop dynamics of the system are oscillatory, with a period similar to the data shown above:

In [ ]:
T = np.linspace(0, 100, 500)
response = ct.input_output_response(
    predprey, T, 0, [35, 35]
)
ct.time_response_plot(response, plot_inputs=False, overlay_signals=True);

We can also visualize the data using a phase plane plot:

In [ ]:
# Generate a simple phase portrait
ct.phase_plane_plot(predprey, [0, 120, 0, 100], 1, gridtype='meshgrid');

We see that the default parameters give a lot of warning messages and the phase portrait does not convey all of the details in some regions of the state space.

We can make sure of some of the functions in the `phaseplot` module to get a better view of the dynamics:

In [ ]:
# Generate a phase portrait
ct.phaseplot.equilpoints(predprey, [-5, 126, -5, 100])
ct.phaseplot.streamlines(
    predprey, np.array([
        [0, 100], [1, 0],
    ]), 10, color='b')
ct.phaseplot.streamlines(
    predprey, np.array([[124, 1]]), np.linspace(0, 10, 500), color='b')
ct.phaseplot.streamlines(
    predprey, np.array([[125, 25], [125, 50], [125, 75]]), 3, color='b')
ct.phaseplot.streamlines(predprey, np.array([2, 8]), 6, color='b')
ct.phaseplot.streamlines(
    predprey, np.array([[20, 30]]), np.linspace(0, 65, 500),
    gridtype='circlegrid', gridspec=[2, 1], arrows=10, color='r')
ct.phaseplot.vectorfield(predprey, [5, 125, 5, 100], gridspec=[20, 20])

# Add the limit cycle
resp1 = ct.initial_response(predprey, np.linspace(0, 100), [20, 75])
resp2 = ct.initial_response(
    predprey, np.linspace(0, 20, 500), resp1.states[:, -1])
plt.plot(resp2.states[0], resp2.states[1], color='k');

### Find the equilibrium points and check stability

We see that there are three equilibrium points in the system.  We can test the stability of the center equilibrium point, which from the phase portrait appears to be unstable.

In [ ]:
xe, ue = ct.find_eqpt(predprey, [20, 30], 0)
print(f"{xe=}")
print(f"{ue=}")

In [ ]:
sys = predprey.linearize(xe, ue)
print(sys)
print("Poles: ", sys.poles())

## Stabilization

Suppose now that we have the ability to modulate the food supply for the hares.  We do this by modifying the parameter $r$ in the model (this is the term `u` in the model at the top of the notebook).  We can use the `place` command to find a set of gains that stabilize the dynamics around the unstable equilibrium point.

In [ ]:
K = ct.place(sys.A, sys.B, [-0.1, -0.2])
print(f"{K=}")

In [ ]:
# Design an eigenvalue placement (EP) controller to stabilize the equilibrium point
epctrl = ct.nlsys(
    None, lambda t, x, u, params: -K @ (u[0:2] - xe),
    inputs=['H', 'L', 'r'], outputs=['u'],
)
predprey_ep = ct.interconnect(
    [predprey, epctrl], inputs=['r'], outputs=['H', 'L', 'u'],
    name='predprey w/ eval placement'
)
print(predprey_ep)

# Show the connection table, useful for debugging what is connected to what
predprey_ep.connection_table()

In [ ]:
xe_ep, ue_ep = ct.find_eqpt(predprey_ep, [20, 30], [0])
print(f"{xe_ep=}")
print(f"{ue_ep=}")
print("Poles: ", predprey_ep.linearize(xe_ep, ue_ep).poles())

In [ ]:
# Generate a simple phase portrait
ct.phase_plane_plot(
    predprey_ep, [0, 120, 0, 100], 1,
    plot_separatrices=False,
    gridtype='meshgrid', gridspec=[8, 5]
    );
ct.phaseplot.streamlines(
    predprey_ep, np.array([xe_ep]), 20, dir='reverse',
    gridtype='circlegrid', gridspec=[4, 11]);

In [ ]:
# Simulation from someplace nearby
T = np.linspace(0, 40)
response = ct.input_output_response(predprey_ep, T, 0, [35, 35])
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response with eval placement, " +
    f"r = {predprey.params['r']}",
    legend_loc='upper right')
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')

## Integral feedback

Another technique that we will learn about later in the class is integral feedback, which can be used to compensate for modeling uncertainty and constant disturbances.

We start by asking what happens if we change the value for the parameter $r$ from its original value of 1.6 to a new value of 1.65 (a change of less than 4%):

In [ ]:
# Simulate with a change in food for the hares
T = np.linspace(0, 40)
response = ct.input_output_response(
    predprey_ep, T, 0, [35, 35], params={'r': 1.65}
)
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response w/ eval placement, " +
    f"r = {response.params['r']}")
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')
response.sysname

We see that the controller no longer stabilizes the equilibrium point (shown with the dashed lines).  In particular, the steady state value of the lynx population does to almost twice the original value.

This effect is even worse if we increase $r$ just a bit more (from 1.65 to 1.7).

In [ ]:
T = np.linspace(0, 40)
response = ct.input_output_response(
    predprey_ep, T, 0, xe, params={'r': 1.7}
)
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response for predprey w/ eval placement, " +
    f"r = {response.params['r']}")
plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')
response.sysname

The system dynamics are now oscillatory, indicating that we are no longer stabilizing the desired equilibrium point.  This indicates a lack of robustness in our feedback control system.

We can compensate for the change in the parameter $r$ by making use of integral feedback in our controller.  We will learn more about integral feedback in later lectures, but for now we demonstrate its ability to compensate for errors in our system model.

In [ ]:
# Integral feedback
# Design an eigenvalue placement (EP) controller to stabilize the equilibrium point
Ki = 0.0001
pictrl = ct.nlsys(
    lambda t, x, u, params: u[1] - u[2],
    lambda t, x, u, params: -K @ (u[0:2] - xe) - Ki * x[0],
    inputs=['H', 'L', 'r'], outputs=['u'], states=1,
)
predprey_pi = ct.interconnect(
    [predprey, pictrl], inputs=['r'], outputs=['H', 'L', 'u'],
    name='predprey_pi'
)
print(predprey_pi)

# Simulate with a change in food for the hares
T = np.linspace(0, 100, 500)
response = ct.input_output_response(
    predprey_pi, T, xe[1], [25, 25, 0], params={'r': 1.65})
ct.time_response_plot(
    response, plot_inputs=False, overlay_signals=True,
    title="I/O response w/ integral action, " +
    f"r = {response.params['r']}",
    legend_loc='upper right')

plt.plot([T[0], T[-1]], [0, 0], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[0], xe_ep[0]], 'k--')
plt.plot([T[0], T[-1]], [xe_ep[1], xe_ep[1]], 'k--')

We see that the system is once again stable at the desired equilibrium point!